# Test Frame Sender

This file is used for sending individual frames to the device. It measures the accuracy based on individual frames.

**Note that the software on the MCU must match this, and not classify based on clusters of frames.**

In [3]:
import numpy as np
import pandas as pd
import gc

In [4]:
df = pd.read_csv("test.csv", header=None)

In [5]:
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float32)            # All but last column as float32
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string
del df
gc.collect()

0

In [6]:
dataset = dataset.reshape(dataset.shape[0], 40, 16, 1)
input_shape = dataset[0].shape

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {input_shape}")

Dataset shape: (2605, 40, 16, 1)
Labels shape: (2605,)
Input shape: (40, 16, 1)


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.95, random_state=42, stratify=labels_set)
del dataset, labels_set
gc.collect()

0

In [8]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))

2025-03-26 14:49:14.677589: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-26 14:49:14.738731: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-26 14:49:14.783167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742996954.829299   23283 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742996954.844603   23283 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742996954.935734   23283 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [9]:
def indexOfMax(arr):
    return np.argmax(arr)

In [11]:
import serial
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM2',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=115200,       # Set baud rate to 115200
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1              # Set timeout for reading
)

Configuring serial port...


In [12]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM2 opened at 115200 baud.


In [13]:
def calc_accuracy(y_true, y_pred):
    success = 0
    failure = 0
    for i in range(len(y_pred)):
        if int(y_pred[i]) == int(indexOfMax(y_true[i])):
            success += 1
        else:
            failure += 1
    res = round(success / (success + failure), 4)
    return res

In [14]:
import time
import struct

x = x_test
y = y_test[0:len(x)]
print(x.shape)

results = []
total = len(x)

# Send data
for i in range(total):
    data = x[i].flatten()
    for j in range(len(data)):
        data_to_send = struct.pack('f', data[j])
        ser.write(data_to_send)
    time.sleep(0.01)
    # Read response
    response = ser.readline().decode('utf-8').strip()
    results.append(response)
    print(f"Guess: {response} - Class: {indexOfMax(y[i])} Accuracy: {calc_accuracy(y, results)} ({i}/{total})")


(2475, 40, 16, 1)
Guess: 0 - Class: 0 Accuracy: 1.0 (0/2475)
Guess: 0 - Class: 0 Accuracy: 1.0 (1/2475)
Guess: 0 - Class: 0 Accuracy: 1.0 (2/2475)
Guess: 2 - Class: 3 Accuracy: 0.75 (3/2475)
Guess: 0 - Class: 0 Accuracy: 0.8 (4/2475)
Guess: 0 - Class: 0 Accuracy: 0.8333 (5/2475)
Guess: 0 - Class: 0 Accuracy: 0.8571 (6/2475)
Guess: 0 - Class: 0 Accuracy: 0.875 (7/2475)
Guess: 1 - Class: 2 Accuracy: 0.7778 (8/2475)
Guess: 0 - Class: 0 Accuracy: 0.8 (9/2475)
Guess: 0 - Class: 0 Accuracy: 0.8182 (10/2475)
Guess: 0 - Class: 0 Accuracy: 0.8333 (11/2475)
Guess: 2 - Class: 3 Accuracy: 0.7692 (12/2475)
Guess: 2 - Class: 0 Accuracy: 0.7143 (13/2475)
Guess: 1 - Class: 3 Accuracy: 0.6667 (14/2475)
Guess: 0 - Class: 0 Accuracy: 0.6875 (15/2475)
Guess: 0 - Class: 0 Accuracy: 0.7059 (16/2475)
Guess: 0 - Class: 0 Accuracy: 0.7222 (17/2475)
Guess: 0 - Class: 0 Accuracy: 0.7368 (18/2475)
Guess: 0 - Class: 0 Accuracy: 0.75 (19/2475)
Guess: 0 - Class: 0 Accuracy: 0.7619 (20/2475)
Guess: 1 - Class: 0 Accur

KeyboardInterrupt: 

In [ ]:
# Close the serial port
ser.close()